# Using multiprocessing to batch compute a multipositioned experiment

Am yet to run time tests on this to see if it works quantitatively better but seems like it does

In [1]:
from multiprocessing import Pool
import tqdm.auto as tqdm
import os
import glob
import numpy as np
from scipy import ndimage as nd
from skimage.io import imsave
from octopuslite import DaskOctopusLiteLoader
from stardist.models import StarDist2D 
from stardist.plot import render_label
from csbdeep.utils import normalize

# CPU count

In [2]:
cpus = os.cpu_count()
cpus

12

# Define segment function

In [3]:
def segment(pos):
#     if not overwrite and glob.glob(f'{root_dir}/{expt}/{pos}/{pos}_images/*channel099*.tif'):
#         print(f'{root_dir}/{expt}/{pos}/{pos}_images/ mask files found, skipping {expt}/{pos}')
#         return
    print('Starting experiment position:', expt, pos)
    # load images
    image_path = f'{root_dir}/{expt}/{pos}/{pos}_images'
    images = DaskOctopusLiteLoader(image_path, 
                                   remove_background = False)
    # iterate over images filenames 
    for frame, fn in (enumerate(images.files(reference_channel))):
        # compile 2-channels into XYC array
        img = np.zeros((images[reference_channel][frame].shape[0], 
                        images[reference_channel][frame].shape[1], 2)) 
        img[:,:,0] = images['irfp'][frame]
        img[:,:,1] = images['rfp'][frame]
        # predict labels using 2ch image
        labels, details = model.predict_instances(normalize(img))
        # format 2channel mask image 
        mask = np.zeros(labels.shape)
        # make multiclass mask from details dict
        for cell_ID, class_label in enumerate(details['class_id'], 1):
            #needs erosion step to stop merging of labels
            segment = nd.binary_erosion(labels==cell_ID)
            mask[segment] = class_label
            # set background to zero
            mask[labels == 0] = 0
        # set filename as mask format (channel099)
        fn = ((images.files('irfp')[frame])).replace('channel003', 'channel099')
        # save out labelled image
        imsave(fn, mask.astype(np.uint8), check_contrast=False)
    return

# Define necessary parameters

In [5]:
from natsort import natsorted

In [6]:
root_dir = '/home/nathan/data/kraken/ras'
expt_list = ['ND0011, ND0010']
pos_list = 'all'
reference_channel = 'irfp'
overwrite = True
expt = 'ND0010'
model = StarDist2D(None, name='stardist_multiclass_MDCK', basedir='../models')
pos_list = natsorted([pos for pos in os.listdir(f'{root_dir}/{expt}') 
               if 'Pos' in pos 
               and os.path.isdir(f'{root_dir}/{expt}/{pos}')] ) 
print('Positions to iterate over:', pos_list)

Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.552369, nms_thresh=0.3.
Positions to iterate over: ['Pos0', 'Pos1', 'Pos2', 'Pos3', 'Pos4', 'Pos5', 'Pos6', 'Pos7', 'Pos8', 'Pos9', 'Pos10', 'Pos11']


# Start parallel processing

In [7]:
pool = Pool(6)

Process ForkPoolWorker-5:
Process ForkPoolWorker-4:
Process ForkPoolWorker-6:
Process ForkPoolWorker-3:
Process ForkPoolWorker-2:
Traceback (most recent call last):
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/multiprocessing/pool.py", line 114, in worker
    task = get()
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/multiprocessing/queues.py", line 365, in get
    with self._rlock:
  File "/home/nathan/analysis/miniconda3/envs/cellx/lib/python3.9/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/nathan/analysis/minic

In [8]:
if __name__ == '__main__':
    with Pool(cpus) as p:
        p.map(segment, pos_list)

Starting experiment position:Starting experiment position:Starting experiment position:Starting experiment position: Starting experiment position:Starting experiment position:Starting experiment position:Starting experiment position:Starting experiment position:Starting experiment position:Starting experiment position: Starting experiment position: ND0010   ND0010     ND0010ND0010 ND0010  ND0010ND0010 ND0010ND0010 ND0010     ND0010ND0010 Pos4Pos7Pos1Pos8Pos2 Pos9Pos5 Pos11


Pos0


Pos10


Pos3

Pos6


Process ForkPoolWorker-17:


KeyboardInterrupt: 

In [ ]:

def info(title):
    print(title)
    print('module name:', __name__)
    print('parent process:', os.getppid())
    print('process id:', os.getpid())

# def f(name):
#     info('function f')
#     print('hello', name)

if __name__ == '__main__':
    info('main line')
    with Pool(cpus) as p:
        p.map(segment, pos_list)
#     p.start()
#     p.join()

main line
module name: __main__
parent process: 490424
process id: 556479
Starting experiment position:Starting experiment position:Starting experiment position:Starting experiment position:Starting experiment position:Starting experiment position:Starting experiment position:Starting experiment position:Starting experiment position:Starting experiment position:      ND0010 ND0010  ND0010ND0010ND0010ND0010  ND0010 ND0010 ND0010  ND0010 Pos4  Pos0Pos11 Pos2 Pos5Pos10Pos9
Pos1
Pos3
Pos8






Starting experiment position: Starting experiment position:ND0010  ND0010 Pos6
Pos7


In [25]:
with Pool(cpus) as p:
        p.map(segment, pos_list)

/home/nathan/data/kraken/ras/ND0010_test/Pos5/Pos5_images/ mask files found, skipping ND0010_test/Pos5
/home/nathan/data/kraken/ras/ND0010_test/Pos6/Pos6_images/ mask files found, skipping ND0010_test/Pos6
/home/nathan/data/kraken/ras/ND0010_test/Pos4/Pos4_images/ mask files found, skipping ND0010_test/Pos4


Source: https://towardsdatascience.com/parallelize-your-python-code-to-save-time-on-data-processing-805934b826e2